## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## First, I'll compute the camera calibration using chessboard images

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
%matplotlib inline

camera_model = None

In [ ]:
class CameraModel():
    def __init__(self, calibration_images):

        mtx, dist = self._calibrate_camera(calibration_images)
        self._camera_matrix = mtx
        self._distortion = dist
        
    def undistort(self, image):
        return cv2.undistort(image, self._camera_matrix, self._distortion, None, self._camera_matrix)
        
    def _calibrate_camera(self, calibration_images):
        objpoints = [] # 3d points in real world space
        imgpoints = [] # 2d points in image plane

        nx = 9 # num corners on the x axis
        ny = 6 # num corners on the y axis

        # Prepare object points: (0,0,0), (1,0,0), (2,0,0) ... (8,5,0)
        objp = np.zeros((ny * nx, 3), np.float32)
        objp[:,:2] = np.mgrid[0:nx, 0:ny].T.reshape(-1, 2) # gen x,y coordinates

        for file in calibration_images:
            img = mpimg.imread(file)

            gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
            ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)

            if ret:
                imgpoints.append(corners)
                objpoints.append(objp)
            else:
                print("Failed to find corners in", file)

        # Now use calibrate camera to get the intrinsics (and ignore extrinsics)
        ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

        return mtx, dist

cal_images = glob.glob('./camera_cal/calibration*.jpg')

if camera_model is None:
    camera_model = CameraModel(cal_images)    

# Now test the distortion
test_image = mpimg.imread('./camera_cal/calibration1.jpg')
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
fig.tight_layout()
ax1.imshow(test_image)
ax1.set_title('Original Image')
ax2.imshow(camera_model.undistort(test_image))
ax1.set_title('Undistorted Image')


In [ ]:
assert(camera_model is not None)

## Create Thresholded Binary Image

In [ ]:

# Edit this function to create your own pipeline.
def pipeline1(img, s_thresh=(170, 255), sx_thresh=(20, 100)):
    img = np.copy(img)
    # Convert to HSV color space and separate the V channel
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HLS).astype(np.float)
    l_channel = hsv[:,:,1]
    s_channel = hsv[:,:,2]
    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    # Stack each channel
    # Note color_binary[:, :, 0] is all 0s, effectively an all black image. It might
    # be beneficial to replace this channel with something else.
    color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary))
    return color_binary

image = mpimg.imread('./test_images/test1.jpg')

result = pipeline1(transform_perspective(camera_model.undistort(image))
)

# Plot the result
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()

ax1.imshow(image)
ax1.set_title('Original Image', fontsize=40)

ax2.imshow(result)
ax2.set_title('Pipeline Result', fontsize=40)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

In [ ]:
def abs_sobel_thresh(img, orient='x', sobel_kernel=3, thresh=(0, 255)):
    # Calculate directional gradient
    # Apply threshold

    # 1) Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # 2) Take the derivative in x or y given orient = 'x' or 'y'
    x = 1 if orient == 'x' else 0
    y = 1 if orient == 'y' else 0
    sobel = cv2.Sobel(gray, cv2.CV_64F, x, y, ksize=sobel_kernel)
    # 3) Take the absolute value of the derivative or gradient
    abs_sobel = np.absolute(sobel)
    # 4) Scale to 8-bit (0 - 255) then convert to type = np.uint8
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # 5) Create a mask of 1's where the scaled gradient magnitude 
            # is > thresh_min and < thresh_max
    binary_output = np.zeros_like(scaled_sobel)
    binary_output[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    # 6) Return this mask as your binary_output image
    return binary_output

def mag_thresh(image, sobel_kernel=3, mag_thresh=(0, 255)):
    """
    Calculate gradient magnitude

    The magnitude, or absolute value, of the gradient is just the square root of
    the squares of the individual x and y gradients. For a gradient in both the
    x and y directions, the magnitude is the square root of the sum of the
    squares.
    """
    # 1) Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    # 2) Take the gradient in x and y separately
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # 3) Calculate the magnitude 
    mag = np.sqrt(np.power(sobelx, 2) + np.power(sobely, 2))
    # 4) Scale to 8-bit (0 - 255) and convert to type = np.uint8
    scaled_sobel = np.uint8(255*mag/np.max(mag))
    # 5) Create a binary mask where mag thresholds are met
    binary_output = np.zeros_like(scaled_sobel)
    binary_output[(scaled_sobel >= mag_thresh[0]) & (scaled_sobel <= mag_thresh[1])] = 1
    # 6) Return this mask as your binary_output image
    return binary_output

def dir_threshold(image, sobel_kernel=3, thresh=(0, np.pi/2)):
    # Calculate gradient direction
    # Apply threshold

    # 1) Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    # 2) Take the gradient in x and y separately
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    abs_sobelx = np.absolute(sobelx)
    abs_sobely = np.absolute(sobely)

    # 4) Use np.arctan2(abs_sobely, abs_sobelx) to calculate the direction of the gradient 
    gradient_dir = np.arctan2(abs_sobely, abs_sobelx)
    
    # 5) Create a binary mask where direction thresholds are met
    binary_output = np.zeros_like(gradient_dir)
    binary_output[(gradient_dir >= thresh[0]) & (gradient_dir <= thresh[1])] = 1
    # 6) Return this mask as your binary_output image
    return binary_output

def hls_select(img, thresh=(0, 255)):
    # 1) Convert to HLS color space
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    S = hls[:,:,2]
    # 2) Apply a threshold to the S channel
    binary_output = np.zeros_like(S)
    binary_output[(S > thresh[0]) & (S <= thresh[1])] = 1
    # 3) Return a binary image of threshold result
    return binary_output

def pipeline(img, s_thresh=(180, 255), sx_thresh=(20, 100)):
    img = np.copy(img)
    # Convert to HSV color space and separate the V channel
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HLS).astype(np.float)
    h_channel = hsv[:,:,0]
    l_channel = hsv[:,:,1]
    s_channel = hsv[:,:,2]
    # Sobel x
    sobelx = cv2.Sobel(s_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    dir_binary = dir_threshold(img, sobel_kernel=5, thresh=(0.7, 1.3))

    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1

    binary =  np.zeros_like(s_channel)
    binary[((dir_binary == 1) & (sxbinary == 1)) | (s_binary == 1)] = 1

    mag_binary = mag_thresh(img, sobel_kernel=3, mag_thresh=(150, 255))
    binary2 =  np.zeros_like(s_channel)
    binary2[(((dir_binary == 1)) & (mag_binary == 1) | (s_binary == 1))] = 1
    
    binary3 =  np.zeros_like(s_channel)
    binary3[((sxbinary == 1)) | (s_binary == 1)] = 1
    
    return binary, binary2, binary3

def combine_thresholds(image):
    # Apply each of the thresholding functions
    ksize=3
    gradx = abs_sobel_thresh(image, orient='x', sobel_kernel=ksize, thresh=(20, 100))
    '''
    grady = abs_sobel_thresh(image, orient='y', sobel_kernel=ksize, thresh=(20, 100))
    mag_binary = mag_thresh(image, sobel_kernel=ksize, mag_thresh=(100, 200))
    dir_binary = dir_threshold(image, sobel_kernel=ksize, thresh=(0.7, 1.3))
    '''
    hls_binary = hls_select(image, thresh=(90, 255))
    combined = np.zeros_like(gradx)
    #combined[((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1))] = 1
    combined[(gradx == 1) | (hls_binary == 1)] = 1
    return combined

test_image = mpimg.imread('./test_images/test1.jpg')
#combined = combine_thresholds(test_image)
combined, combined2, combined3 = pipeline(test_image)
# Plot the result
f, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(test_image)
ax1.set_title('Original Image', fontsize=50)
ax2.imshow(combined, cmap='gray')
ax2.set_title('Combined Thresholds', fontsize=50)
ax3.imshow(combined2, cmap='gray')
ax3.set_title('Combined Thresholds', fontsize=50)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
ax4.imshow(combined3, cmap='gray')
ax4.set_title('Combined Thresholds', fontsize=50)

## Perspective Transform

In [ ]:
def get_perspective_transform_func():
    # src and dst points found externally
    src = np.float32([[581, 460], [702, 460], [1017, 665], [285, 665]])
    dst = np.float32([[250, 0], [1046, 0], [1046, 708], [250, 708]])
    
    M = cv2.getPerspectiveTransform(src, dst)
    
    def apply_perspective_transform(img):
        height, width = img.shape[:2]
        return cv2.warpPerspective(img, M, (width, height), flags=cv2.INTER_LINEAR)
    
    return apply_perspective_transform

transform_perspective = get_perspective_transform_func()

test_image1 = mpimg.imread('./test_images/straight_lines1.jpg')

top_down1 = transform_perspective(camera_model.undistort(test_image1))
f, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(test_image1)
ax1.set_title('Original Image', fontsize=50)
ax2.imshow(top_down1)
ax2.set_title('Undistorted and warped', fontsize=50)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

test_image2 = mpimg.imread('./test_images/straight_lines2.jpg')
top_down2 = transform_perspective(camera_model.undistort(test_image2))
ax3.imshow(test_image2)
ax3.set_title('Original Image', fontsize=50)
ax4.imshow(top_down2)
ax4.set_title('Undistorted and warped', fontsize=50)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

